In [2]:
cd ~/Documents/Galvanize/mtg-capstone/src/

/Users/benjaminwalzer/Documents/Galvanize/mtg-capstone/src


In [3]:
import requests
import psycopg2
import os
import numpy as np
from deck_scraping import format_deck, ReflexiveDict, parse_card_string

In [4]:
card_dict = ReflexiveDict()

In [5]:
raw_deck_list = '''4 Sakura-Tribe Elder
                4 Primeval Titan
                2 Sweltering Suns
                4 Search for Tomorrow
                4 Scapeshift
                2 Hour of Promise
                4 Farseek
                2 Explore
                2 Summoner's Pact
                3 Lightning Bolt
                2 Prismatic Omen
                4 Wooded Foothills
                3 Windswept Heath
                4 Valakut, the Molten Pinnacle
                4 Stomping Ground
                3 Cinder Glade
                6 Mountain
                2 Forest
                1 Wood Elves
                Sideboard
                2 Obstinate Baloth
                2 Chameleon Colossus
                3 Relic of Progenitus
                1 Engineered Explosives
                2 Anger of the Gods
                1 Nature's Claim
                2 Beast Within
                1 Ancient Grudge
                1 Reclamation Sage
                '''

In [45]:
with open('/Users/benjaminwalzer/Downloads/cardstorm_test_kanji_4.txt', 'r') as f:
    deck_list = f.read()

In [46]:
deck_list = deck_list.split('\n')

In [47]:
all_cardstorm_ids = card_dict.get_cardstorm_ids()

In [48]:
def deck_to_dict(deck_list):
    deck_dict = {}
    for card_string in deck_list:
        if card_string.startswith('S'):
            break
        if not card_string:
            break
        parse_status, card_name, card_count = parse_card_string(card_string)
        cardstorm_id = card_dict[card_name.lower()]
        deck_dict[cardstorm_id] = int(card_count)
        
    return deck_dict

In [49]:
deck_dict = deck_to_dict(deck_list)

In [50]:
# need to make a 11,438 x 1 vector of this deck list
deck_vector = []
for cardstorm_id in all_cardstorm_ids:
    if cardstorm_id in deck_dict:
        deck_vector.append(deck_dict[cardstorm_id])
    else:
        deck_vector.append(0)
deck_vector = np.array(deck_vector)

In [51]:
# retrieve the product_features matrix from the db

In [52]:
dbname = os.environ['CAPSTONE_DB_DBNAME']
host = os.environ['CAPSTONE_DB_HOST']
username = os.environ['CAPSTONE_DB_USERNAME']
password = os.environ['CAPSTONE_DB_PASSWORD']

In [53]:
conn = psycopg2.connect('dbname={} host={} user={} password={}'.format(dbname, host, username, password))
cursor = conn.cursor()

In [54]:
query = '''SELECT cardstorm_id, features
           FROM product_matrices
           WHERE date = (SELECT MAX(date) FROM product_matrices)
           ORDER BY cardstorm_id ASC'''

cursor.execute(query)

In [55]:
results = cursor.fetchall()

In [56]:
feature_matrix = []
for cardstorm_id, features in results:
    feature_matrix.append(features)
    
feature_matrix = np.array(feature_matrix)

In [57]:
feature_matrix.shape

(11348, 10)

In [70]:
result = np.linalg.lstsq(feature_matrix, deck_vector)

In [71]:
result[0]

array([-0.15642874,  0.06220244, -0.31110072,  0.00287821,  0.44539073,
        0.08936423,  0.10256757, -0.01984965, -0.10238489, -0.29790892])

In [60]:
result[1]

array([ 53.44402113])

In [77]:
recreated = np.dot(result[0], feature_matrix.T)
print(recreated)

[  4.33747214e-07   4.33747214e-07   4.33747214e-07 ...,   4.33747214e-07
   4.33747214e-07   3.56760343e-04]


In [143]:
np.argsort(recreated - deck_vector)[:10]

array([ 9211,  3790,  4250,  2333,   101,   994,   948,  9512, 10011,  6346])

In [63]:
all_cardstorm_ids = np.array(all_cardstorm_ids)

In [139]:
for idx, cardstorm_id in enumerate(all_cardstorm_ids[np.argsort(recreated - deck_vector)[::]]):
    print(card_dict[cardstorm_id])
    
    if idx == 9:
        break

sphinx's revelation
glacial fortress
hallowed fountain
detention sphere
aetherling
bloodstained mire
blood crypt
street wraith
thought scour
mutagenic growth


In [154]:
import predictions

In [155]:
with open('/Users/benjaminwalzer/Downloads/cardstorm_example_UW.txt', 'r') as f:
    deck_list = f.read()

In [156]:
deck_list

'4 Cryptic Command\n4 Snapcaster Mage\n4 Celestial Colonnade\n4 Path to Exile'

In [157]:
predictions.do_everything(deck_list)

KeyError: 'cryptic command'

In [137]:
result[0] == fn_u

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [126]:
fn_recommendations

array([  4.33747214e-07,   4.33747214e-07,   4.33747214e-07, ...,
         4.33747214e-07,   4.33747214e-07,   3.56760343e-04])

In [127]:
fn_top == np.argsort(recreated - deck_vector)[:-10:-1]

array([False, False, False, False, False, False, False, False, False], dtype=bool)

In [128]:
card_dict[7022]

'path to exile'

In [129]:
cursor

<cursor object at 0x1093bb428; closed: 0>

In [159]:
bool(np.array([1,2,3]).)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [160]:
test = np.array([1,2,3])

In [161]:
test.all()

True

In [162]:
test2 = np.array([])

In [163]:
test2

True